In [ ]:
from sciencecrawler.pubmed import PubmedSearch
import json
from neo4jsciencecrawler import Neo4jScienceCrawler
import time

In [ ]:
words_covid = ["COVID-19","SARS‑CoV‑2","2019-nCoV","HCoV-19","hCoV-19","2019-nCoV","Wuhan coronavírus","Wuhan pneumonia"]


In [ ]:
searchengine = PubmedSearch('covid chloroquine')

In [ ]:
total = searchengine.total_search

In [ ]:
total

In [ ]:
articles_current_page = searchengine.get_list_articles()

In [ ]:
articles_read = len(articles_current_page)

In [ ]:
neo4jC = Neo4jScienceCrawler("bolt://localhost:7687",'neo4j','admin')

In [ ]:
def check_containers_words(abstract):
    global words_covid
    abstract_lower = abstract.lower()
    for word in words_covid:
        if word.lower() in abstract_lower:
            return True
    return False


In [ ]:
def save_in_neo4j(article):
    article_json = article.to_json()
    try:
        references = article.get_references()
        article_json["is_covid"] = check_containers_words(article['abstract'])
        neo4jC.create_article(article_json)
        references = article.get_references()
        for ref in references:
            ref_json = ref.to_json()
            neo4jC.create_connection_ref(article_json['id'],ref_json)
    except Exception as ex:
        print(ex)

In [ ]:
for article in articles_current_page:
    print(article.url)
    save_in_neo4j(article)

In [ ]:
while articles_read < total:
    time.sleep(5)
    print(articles_read)
    searchengine.get_next_page()
    articles_current_page = searchengine.get_list_articles()
    articles_read = articles_read + len(articles_current_page)
    for article in articles_current_page:
        try:
            print(article.url)
            save_in_neo4j(article)
        except Exception as ex:
            print(ex)
      